In [1]:
import gpytorch
import numpy as np
import torch
import winsound
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from LDGD.model import LDGD, FastLDGD, VAE
from LDGD.visualization.vizualize_utils import plot_heatmap, plot_2d_scatter, plot_ARD_gplvm
from LDGD.visualization.vizualize_utils import plot_loss_gplvm, plot_scatter_gplvm, plot_box_plots
from LDGD.data.data_loader import generate_data

from gpytorch.likelihoods import GaussianLikelihood, BernoulliLikelihood

from LDGD.utils import dicts_to_dict_of_lists
import json
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2
duration = 1000  # milliseconds
freq = 440  # Hz
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Compare_decoding_on_dataset.ipynb# Functions

## 1- Create Dataset

In [2]:
def create_dataset(num_dimension, random_state, test_size, **kwargs):
    # Extract parameters for synthetic data generation
    pattern = kwargs.get('pattern', 'moon')  # default pattern
    n_samples = kwargs.get('n_samples', 1500)
    noise = kwargs.get('noise', 0.1)
    increase_method = kwargs.get('increase_method', 'linear')

    X, y, orig_data = generate_data(pattern, n_samples, noise, num_dimension, increase_method, random_state=random_state)
    # One-hot encode the labels
    y_one_hot = np.zeros((y.shape[0], len(np.unique(y))))
    y_one_hot[np.arange(y.shape[0]), np.uint(y)] = 1

    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test, y_train_labels, y_test_labels = train_test_split(X, y_one_hot, y,
                                                                                       test_size=test_size,
                                                                                       random_state=random_state)
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    y_train_labels_tensor = torch.tensor(y_train_labels)
    y_test_labels_tensor = torch.tensor(y_test_labels)

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, y_train_labels_tensor, y_test_labels_tensor, orig_data



## 2 - Create Model

In [3]:
def create_LDGD_model(data_cont, data_cat, ldgd_settings, batch_shape, x_init='pca'):
    kernel_reg = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=ldgd_settings['latent_dim']))
    kernel_cls = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=ldgd_settings['latent_dim']))

    likelihood_reg = GaussianLikelihood(batch_shape=batch_shape)
    likelihood_cls = BernoulliLikelihood()
    model = LDGD(data_cont,
                 kernel_reg=kernel_reg,
                 kernel_cls=kernel_cls,
                 num_classes=data_cat.shape[-1],
                 latent_dim=ldgd_settings['latent_dim'],
                 num_inducing_points_cls=ldgd_settings['num_inducing_points_cls'],
                 num_inducing_points_reg=ldgd_settings['num_inducing_points_reg'],
                 likelihood_reg=likelihood_reg,
                 likelihood_cls=likelihood_cls,
                 use_gpytorch=ldgd_settings['use_gpytorch'],
                 shared_inducing_points=ldgd_settings['shared_inducing_points'],
                 use_shared_kernel=False,
                 x_init=x_init,
                 device=device)

    return model

def create_FastLDGD_model(data_cont, data_cat, ldgd_settings, batch_shape):
    kernel_reg = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=ldgd_settings['latent_dim']))
    kernel_cls = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=ldgd_settings['latent_dim']))

    likelihood_reg = GaussianLikelihood(batch_shape=batch_shape)
    likelihood_cls = BernoulliLikelihood()
    model = FastLDGD(data_cont,
             kernel_reg=kernel_reg,
             kernel_cls=kernel_cls,
             num_classes=data_cat.shape[-1],
             latent_dim=ldgd_settings['latent_dim'],
             num_inducing_points_cls= ldgd_settings['num_inducing_points_cls'],
             num_inducing_points_reg= ldgd_settings['num_inducing_points_reg'],
             likelihood_reg=likelihood_reg,
             likelihood_cls=likelihood_cls,
             use_gpytorch=ldgd_settings['use_gpytorch'],
             shared_inducing_points=ldgd_settings['shared_inducing_points'],
             use_shared_kernel=False,
             device=device)

    return model




In [4]:

# Assuming X, y are numpy arrays
def cross_validate_model(n_dim, settings, n_splits=5, load_saved_result=False, save_model=True, **kwargs):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=model_settings['random_state'])

    performances = []

    for n_dim in n_dim:
        # Code to generate data for different dimensional synthetic datasets
        pattern = kwargs.get('pattern', 'moon')  # default pattern
        n_samples = kwargs.get('n_samples', 1500)
        noise = kwargs.get('noise', 0.1)
        increase_method = kwargs.get('increase_method', 'linear')

        X, y, orig_data = generate_data(pattern, n_samples, noise, n_dim, increase_method,
                                        random_state=model_settings['random_state'])
        # One-hot encode the labels
        y_one_hot = np.zeros((y.shape[0], len(np.unique(y))))
        y_one_hot[np.arange(y.shape[0]), np.uint(y)] = 1

        # Convert to PyTorch tensors
        X = torch.tensor(X, dtype=torch.float32)
        labels = torch.tensor(y, dtype=torch.float32)
        y = torch.tensor(y_one_hot)


        p, r, f = [], [], []
        for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(X, labels)):
            print(f"=============== Dim {n_dim} Fold {fold_idx} ===============")
            # "X[train_ix]" will have to be modified to extract the relevant synthetic rows
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            labels_train, labels_test = labels[train_idx], labels[test_idx]

            # Here you convert to PyTorch tensors, etc., and pass into your network
            # metrics = train_and_evaluate_model() assuming it returns a dict or other
            # performance data. Replace with the appropiate flow of your process.
            settings['data_dim'] = X_train.shape[-1]
            batch_shape = torch.Size([settings['data_dim']])
            if load_saved_result is False:
                model = create_LDGD_model(X_train, y_train, settings, batch_shape=batch_shape, x_init='pca')
                losses, history_train = model.train_model(yn=X_train, ys=y_train,
                                                          epochs=settings['num_epochs_train'],
                                                          batch_size=settings['batch_size'],
                                                          verbos=0)
                if save_model is True:
                    model.save_wights(path_save='./saved_models/', file_name=f"model_synthetic_fold{fold_idx}_synthetic{2*n_dim}")
                predictions, metrics, history_test = model.evaluate(yn_test=X_test, ys_test=labels_test,
                                                                    epochs=settings['num_epochs_test'],
                                                                    verbos=0)

                winsound.Beep(freq, duration)
            else:
                model = create_LDGD_model(X_train, y_train, settings)
                model.load_weights(path_save='./saved_models/', file_name=f'model_synthetic_fold{fold_idx}_synthetic{2*n_dim}.pth')
                predictions, metrics, history_test = model.evaluate(yn_test=X_test, ys_test=labels_test,
                                                                    epochs=settings['num_epochs_test'],
                                                                    verbos=0)

            # Compute or accumulate measures (precision, recall, F1 score)
            p.append(metrics['precision'])
            r.append(metrics['recall'])
            f.append(metrics['f1_score'])

        # Form the list of dataframe rows for the round of n_dim rounds
        measures = (np.mean(p), np.std(p), np.mean(r), np.std(r), np.mean(f), np.std(f))
        round_result = {"dataset": f"synthetic {2*n_dim}", "precision": f"{measures[0]:.2f} ± {measures[1]:.2f}",
                        "recall": f"{measures[2]:.2f} ± {measures[3]:.2f}",
                        "f score": f"{measures[4]:.2f} ± {measures[5]:.2f}"}
        performances.append(round_result)

    # Creating DataFrame from performance data
    df_performances = pd.DataFrame(performances)
    print(df_performances)
    return df_performances

# Task 5: Compare LDGD Results with VAE
## 5.1 Settings:

In [5]:
model_settings = {
    'latent_dim': 2,
    'num_inducing_points_reg': 5,
    'num_inducing_points_cls': 5,
    'num_epochs_train': 10000,
    'num_epochs_test': 5000,
    'batch_size': 100,
    'load_trained_model': False,
    'load_tested_model': False,
    'dataset': 'synthetic',
    'shared_inducing_points': True,
    'use_gpytorch': True,
    'random_state': 54,
    'test_size': 0.3,
    'use_shared_kernel': False
}

## 3.2 Create Dataset

In [6]:
yn_train, yn_test, ys_train, ys_test, labels_train, labels_test, orig_dataset = create_dataset(num_dimension=10,
                                                                                               random_state= model_settings['random_state'],
                                                                                               test_size=model_settings['test_size'])

print(f"train size = {yn_train.shape[0]}")
print(f"test size = {ys_test.shape[0]}")

list_size = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
load_saved_result = False


## 3.3 Training model
# VAE

In [9]:
try:
    with open('metric_vae_dict.json', 'r') as file:
        metric_vae_dict = json.load(file)
except:
    batch_shape = torch.Size([yn_train.shape[-1]])
    metric_vae_list = []
    for train_size in list_size:
        print(f"Training for sample size: {train_size}")
        yn_train_limited, ys_train_limited, labels_train_limited = yn_train[:train_size], ys_train[:train_size], labels_train[:train_size]

        # Train VAE Model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = VAE(input_dim=yn_train_limited.shape[-1],
                    hidden_dim=30,
                    latent_dim=10,
                    num_classes=len(np.unique(labels_train))).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        model.fit(x=yn_train_limited, y=labels_train_limited, x_test=yn_test, y_test=labels_test, optimizer=optimizer, epochs=2000, batch_size=500, patience=20)
        x_hat, y_hat, mean, log_var, metrics = model.evaluate(yn_test, labels_test)

        metric_vae_list.append(metrics)
    winsound.Beep(freq, duration*3)
    metric_vae_dict = dicts_to_dict_of_lists(metric_vae_list)
    with open('metric_vae_dict.json', 'w') as file:
        json.dump(metric_vae_dict, file, indent=4)

# Train LDGD

In [7]:
try:
    with open('./saved_results/metric_ldgd_dict.json', 'r') as file:
        metric_ldgd_dict = json.load(file)
except:
    batch_shape = torch.Size([yn_train.shape[-1]])
    metric_ldgd_list = []
    for train_size in list_size:
        model_settings['num_inducing_points_reg']= np.max([int(train_size/10) + 5, 25])
        model_settings['num_inducing_points_cls']= np.max([int(train_size/10) + 5, 25])
        yn_train_limited, ys_train_limited, labels_train_limited = yn_train[:train_size], ys_train[:train_size], labels_train[:train_size]
        # Train LDGD Model
        model = create_LDGD_model(yn_train_limited, ys_train_limited, model_settings)
        if load_saved_result is False:
            losses, history_train = model.train_model(yn=yn_train_limited, ys=ys_train_limited,
                                                      epochs=model_settings['num_epochs_train'],
                                                      batch_size=model_settings['batch_size'],
                                                      early_stop=1e-6)
            model.save_wights(path_save='', file_name=f"model_ldgd_task3_{train_size}")
        else:
            model.load_weights(path_save='', file_name=f"model_ldgd_task3_{train_size}")

        predictions, metrics, history_test = model.evaluate(yn_test=yn_test, ys_test=labels_test,
                                                        epochs=model_settings['num_epochs_test'])

        metric_ldgd_list.append(metrics)
    winsound.Beep(freq, duration*3)
    metric_ldgd_dict = dicts_to_dict_of_lists(metric_ldgd_list)
    with open('metric_ldgd_dict.json', 'w') as file:
        json.dump(metric_ldgd_dict, file, indent=4)

# Train FastLDGD

In [11]:
try:
    with open('./hsaved_results/metric_fastldgd_dict.json', 'r') as file:
        metric_fastldgd_dict = json.load(file)
except:
    batch_shape = torch.Size([yn_train.shape[-1]])
    metric_fastldgd_list = []
    for train_size in list_size:
        print(f"\n ===== train size - {train_size} ======")
        model_settings['num_inducing_points_reg']= np.max([int(train_size/10) + 5, 25])
        model_settings['num_inducing_points_cls']= np.max([int(train_size/10) + 5, 25])
        yn_train_limited, ys_train_limited, labels_train_limited = yn_train[:train_size], ys_train[:train_size], labels_train[:train_size]
        # Train LDGD Model
        model = create_FastLDGD_model(yn_train_limited, ys_train_limited, model_settings, batch_shape=batch_shape)
        if load_saved_result is False:
            losses, history_train = model.train_model(yn=yn_train_limited, ys=ys_train_limited,
                                                      epochs=model_settings['num_epochs_train'],
                                                      batch_size=model_settings['batch_size'])
            model.save_wights(path_save='', file_name=f"model_fldgd_task3_{train_size}")
        else:
            try:
                model.load_weights(path_save='', file_name=f"model_fldgd_task3_{train_size}")
            except:
                losses, history_train = model.train_model(yn=yn_train_limited, ys=ys_train_limited,
                                                          epochs=model_settings['num_epochs_train'],
                                                          batch_size=model_settings['batch_size'])
                model.save_wights(path_save='', file_name=f"model_fldgd_task3_{train_size}")


        predictions, metrics, history_test = model.evaluate(yn_test=yn_test, ys_test=labels_test,
                                                        epochs=model_settings['num_epochs_test'])

        metric_fastldgd_list.append(metrics)
    winsound.Beep(freq, duration*3)
    metric_fastldgd_dict = dicts_to_dict_of_lists(metric_fastldgd_list)
    with open('metric_fastldgd_dict.json', 'w') as file:
        json.dump(metric_fastldgd_dict, file, indent=4)

## Visualize

In [13]:

AXIS_LABEL_FONTSIZE = 32
TICKS_LABEL_FONTSIZE = 28
LEGEND_FONTSIZE = 24



fig, axes = plt.subplots(1,2, figsize=(14, 5))
# ax1.set_title('2D Latent Subspace Corresponding to 3 Phase Oilflow', fontsize=32)

axes[0].plot(list_size, metric_vae_dict['accuracy'], label='VAE', marker='s')
axes[0].plot(list_size, metric_ldgd_dict['accuracy'], label='LDGD', marker='d')
axes[0].plot(list_size, metric_fastldgd_dict['accuracy'], label='FastLDGD', marker='o')

axes[0].set_xlabel(f'Number of data points', fontsize=AXIS_LABEL_FONTSIZE)
axes[0].set_ylabel(f'Accuracy', fontsize=AXIS_LABEL_FONTSIZE)
axes[0].tick_params(axis='both', labelsize=TICKS_LABEL_FONTSIZE)
# Applying consistent spines format
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].spines['left'].set_linewidth(1)
axes[0].spines['bottom'].set_linewidth(1)
#axes[0].legend(fontsize=LEGEND_FONTSIZE)
axes[0].set_xscale('log')  # Making x-axis logarithmic for the accuracy plot


axes[1].plot(list_size, metric_vae_dict['f1_score'], label='VAE', marker='s')
axes[1].plot(list_size, metric_ldgd_dict['f1_score'], label='LDGD', marker='d')
axes[1].plot(list_size, metric_fastldgd_dict['f1_score'], label='Fast LDGD', marker='o')

axes[1].set_xlabel(f'Number of data points', fontsize=AXIS_LABEL_FONTSIZE)
axes[1].set_ylabel(f'F-measure', fontsize=AXIS_LABEL_FONTSIZE)
axes[1].tick_params(axis='both', labelsize=TICKS_LABEL_FONTSIZE)
# Applying consistent spines format
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].spines['left'].set_linewidth(1)
axes[1].spines['bottom'].set_linewidth(1)
#axes[1].legend(fontsize=LEGEND_FONTSIZE)
axes[1].set_xscale('log')  # Making x-axis logarithmic for the accuracy plot

plt.tight_layout()
fig.savefig("./saved_results/figure5_comp_models.png")
fig.savefig("./saved_results/figure5_comp_models.svg")

plt.show()

In [29]:
yn_train, yn_test, ys_train, ys_test, labels_train, labels_test, orig_dataset = create_dataset(num_dimension=10,
                                                                                               random_state= model_settings['random_state'],
                                                                                               test_size=model_settings['test_size'],
                                                                                               n_samples=500)

print(f"train size = {yn_train.shape[0]}")
print(f"test size = {ys_test.shape[0]}")

load_saved_result = False


In [30]:
print(f"Training for sample size: {train_size}")

# Train VAE Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=yn_train.shape[-1],
            hidden_dim=30,
            latent_dim=10,
            num_classes=len(np.unique(labels_train))).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.fit(x=yn_train, y=labels_train, x_test=yn_test, y_test=labels_test, optimizer=optimizer, epochs=2000, batch_size=500, patience=20)
x_hat, y_hat, mean, log_var, metrics = model.evaluate(yn_test, labels_test)



In [31]:
x_train_hat, y_hat, mean_train, log_var_train, metrics = model.evaluate(yn_train, labels_train)


In [34]:

fig, axs = plt.subplots(1,2, figsize=(16, 8))
l1=0
l2=1
plot_scatter_gplvm(mean_train, labels_train, l1=l1, l2=l2, ax=axs[0], colors=['r', 'b', 'g'], show_errorbars=False, std=None)
plot_scatter_gplvm(mean, labels_test, l1=l1, l2=l2, ax=axs[1], colors=['r', 'b', 'g'], show_errorbars=False, std=None)